<a href="https://colab.research.google.com/github/whitestones011/deep_learning/blob/master/pytorch_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
torch.__version__

In [ ]:
!python -V

# Dataset & DataLoader

Custom Dataset class should have implementation of `__init__`, `__len__` and `__get_item__` functions.

In [ ]:
from dataclasses import dataclass, field

import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [ ]:
@dataclass(order=True)
class HousingDataset(Dataset):
  path: str
  data: pd.DataFrame = field(init=False)

  def __post_init__(self):
    self.data = pd.read_csv(self.path).to_numpy()

  def __len__(self):
    """
    Return dataset size
    """
    return self.data.shape[0]

  def __getitem__(self, idx):
    """
    Return features and label for single sample at index idx
    """
    features = self.data[idx,:-1]
    label = self.data[idx,-1]
    return features, label


Creating instance of HousingDataset class.

In [ ]:
train = HousingDataset('/content/sample_data/california_housing_train.csv')
test = HousingDataset('/content/sample_data/california_housing_test.csv')

In [ ]:
train_loader = DataLoader(
    train,
    batch_size=2,
    shuffle=True,
)

In [ ]:
test_loader = DataLoader(
    train,
    batch_size=2,
    shuffle=True,
)

In [ ]:
# load first batch
features, labels = next(iter(train_loader))

In [ ]:
features[0]

In [ ]:
labels.view(-1,1)

# Model

In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define the three linear layers
        self.fc1 = nn.Linear(8, 16, dtype=float)
        self.fc2 = nn.Linear(16, 8, dtype=float)
        self.fc3 = nn.Linear(8, 1, dtype=float)

    def forward(self, x):
        # Pass x through linear layers adding activations
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        return x

In [ ]:
model = Net()

## Training model


In [ ]:
# Define loss function and optimizer
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Iterate over epochs and training batches
for epoch in range(1000):

  for features, labels in train_loader:
    # Clear gradients
    optimizer.zero_grad()

    # Forward pass: get model's outputs
    outputs = model(features)

    # Compute loss
    loss = loss_func(outputs, labels.view(-1,1))

    # Compute gradients
    loss.backward()

    # Optimizer's step: update params(weights)
    optimizer.step()

## Evaluation

In [ ]:
!pip install torchmetrics

In [ ]:
from torchmetrics.regression import MeanSquaredError

In [ ]:
# Set up evaluation metric
mse = MeanSquaredError(num_outputs=2)

model.eval()
with torch.no_grad():
    for features, labels in test_loader:
        # Get predicted prices for test data batch
        outputs = model(features)
        mse(outputs, labels.view(-1, 1))

# Compute total test MSE
test_mse = mse.compute()
print(f"Test MSE: {test_mse}")